In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

2.7.0


In [ ]:
# Folium setup.
import folium
print(folium.__version__)

0.8.3


In [ ]:
from functools import partial
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [ ]:
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers
from tensorflow.python.keras import losses

In [ ]:
# INSERT YOUR BUCKET HERE:
BUCKET = 'tfworkshop'
#BUCKET = 'practice_tanuja'

In [ ]:
from tensorflow.python.keras import backend

def dice_coeff(y_true, y_pred, smooth=1):
    y_true_f = backend.flatten(y_true)
    y_pred_f = backend.flatten(y_pred)
    intersection = backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (backend.sum(y_true_f) + backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

In [ ]:
# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'HKH/Tanuja/New_data8'
TRAINING_BASE = 'training'
VALIDATION_BASE = 'validation'
TESTING_BASE = 'testing'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['blue', 'green', 'red',"vnir","swir","NDVI"]
BANDS = opticalBands
RESPONSE = 'class'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 128
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]


COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))
print(FEATURES_DICT)

#Sizes of the training and evaluation datasets.
TRAIN_SIZE = 4000
EVAL_SIZE = 2000

# Specify model training parameters.
BATCH_SIZE = 8
#EPOCHS = 5
BUFFER_SIZE = 2000
#OPTIMIZER = 'SGD'
OPTIMIZER = 'adam'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']


# BUFFER_SIZE = 2000
# #OPTIMIZER = 'Adam'
# LOSS = dice_loss
# METRICS = [
#     metrics.get('RootMeanSquaredError'),
#     metrics.get('MeanAbsoluteError'),
#     metrics.get('Accuracy'),
#     dice_coeff,
# ]

{'blue': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'green': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'red': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'vnir': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'NDVI': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None), 'class': FixedLenFeature(shape=[128, 128], dtype=tf.float32, default_value=None)}


In [ ]:
def normalized_difference(a, b):
  """Compute normalized difference of two inputs.

  Compute (a - b) / (a + b).  If the denomenator is zero, add a small delta.

  Args:
    a: an input tensor with shape=[1]
    b: an input tensor with shape=[1]

  Returns:
    The normalized difference as a tensor.
  """
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def add_NDVI(features):
  """Add NDVI to the dataset.
  Args:
    features: a dictionary of input tensors keyed by feature name.
    label: the target label

  Returns:
    A tuple of the input dictionary with an NDVI tensor added and the label.
  """
  print('hello')
  features['NDVI'] = normalized_difference(features['vnir'], features['red'])
  return features

# def add_NDBI(features, label):
#   """Add NDVI to the dataset.
#   Args:
#     features: a dictionary of input tensors keyed by feature name.
#     label: the target label

#   Returns:
#     A tuple of the input dictionary with an NDVI tensor added and the label.
#   """
#   features['NDBI'] = normalized_difference(features['swir'], features['vnir'])
#   return features, label


In [ ]:
# Data parsing and preparation 
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)

# def parse_tfrecord(example_proto):
#   """The parsing function.
#   Read a serialized example into the structure defined by FEATURES_DICT.
#   Args:
#     example_proto: a serialized Example.
#   Returns:
#     A dictionary of tensors, keyed by feature name.
#   """
#   parsed_features = tf.io.parse_single_example(example_proto, FEATURES_DICT)
#   labels = parsed_features.pop(RESPONSE)
#   return parsed_features, tf.cast(labels, tf.int32)
  
def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  print(BANDS)
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]



In [ ]:
def get_dataset(pattern,training=False):
    """Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
    Returns: 
    A tf.data.Dataset
    """
    glob = tf.io.gfile.glob(pattern)
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(to_tuple, num_parallel_calls=5)
    if training:
        dataset = dataset.map(transform)
    return dataset

In [ ]:
# custom function to randomly augment the data during training
def transform(features,labels):
    x = tf.random.uniform(())
    # flip image on horizontal axis
    if x < 0.12: 
        feat = tf.image.flip_left_right(features)
        labl = tf.image.flip_left_right(labels)
    # flip image on vertical axis
    elif tf.math.logical_and(x >=0.12, x < 0.24):
        feat = tf.image.flip_up_down(features)
        labl = tf.image.flip_up_down(labels)
    # transpose image on bottom left corner
    elif tf.math.logical_and(x >=0.24, x < 0.36):
        feat = tf.image.flip_left_right(tf.image.flip_up_down(features))
        labl = tf.image.flip_left_right(tf.image.flip_up_down(labels))
    # rotate to the left 90 degrees
    elif tf.math.logical_and(x >=0.36, x < 0.48):
        feat = tf.image.rot90(features,k=1)
        labl = tf.image.rot90(labels,k=1)
    # rotate to the left 180 degrees
    elif tf.math.logical_and(x >=0.48, x < 0.60):
        feat = tf.image.rot90(features,k=2)
        labl = tf.image.rot90(labels,k=2)
    # rotate to the left 270 degrees
    elif tf.math.logical_and(x >=0.60, x < 0.72):
        feat = tf.image.rot90(features,k=3)
        labl = tf.image.rot90(labels,k=3)
    # transpose image on bottom right corner
    elif tf.math.logical_and(x >=0.72, x < 0.84):
        feat = tf.image.flip_left_right(tf.image.rot90(features,k=2))
        labl = tf.image.flip_left_right(tf.image.rot90(labels,k=2))
    else:
        feat = features
        labl = labels
    
    return feat,labl

In [ ]:
def get_training_dataset():
    """Get the preprocessed training dataset
    Returns: 
    A tf.data.Dataset of training data.
    """
    glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '/' + '*'
    #dataset = get_dataset(glob,training=True)
    dataset = get_dataset(glob,training=True)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    return dataset
training = get_training_dataset()

type(training)


In [ ]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '/' + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()
type(training)

In [ ]:
def get_validation_dataset():
	"""Get the preprocessed validaton dataset
  Returns: 
    A tf.data.Dataset of validation data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + VALIDATION_BASE + '/' + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

validation = get_validation_dataset()

['blue', 'green', 'red', 'vnir', 'NDVI']


In [ ]:
def get_testing_dataset():
  glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TESTING_BASE + '/' + '*'
  dataset = get_dataset(glob)
  dataset = dataset.batch(1).repeat()
  return dataset
testing = get_testing_dataset()

['blue', 'green', 'red', 'vnir', 'NDVI']


In [ ]:
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder



In [ ]:
def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
  #layers.concatenate([concat_tensor, decoder], axis=-1)
  #inputs = layers.Concatenate()([, x2])
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER), 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model


In [ ]:
#mount to google drive
from google.colab import drive
drive.mount('/content/mnt')

Mounted at /content/mnt


In [ ]:
%cd /content/mnt/My Drive
%ls

In [ ]:

!mkdir -p 29_09_newdata8_rgbifndvi_datatran_40_epoch


In [ ]:
m = get_model()
m.summary()

history = m.fit(
    x=training, 
    epochs=40, 
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
    validation_data=testing,
    validation_steps=EVAL_SIZE)

m.save('29_09_newdata8_rgbifndvi_datatran_40_epoch/my_model')

In [ ]:
plt.rcParams["figure.figsize"] = (40,10)

In [ ]:
#loading the saved model 
MODEL_DIR = '29_09_newdata8_rgbifndvi_datatran_40_epoch/my_model'

m = tf.keras.models.load_model(MODEL_DIR)
m.summary()

In [ ]:
#moving model to google cloud storage
!gsutil cp -r /content/mnt/MyDrive//29_09_newdata8_rgbifndvi_datatran_40_epoch gs://tfworkshop/HKH/Tanuja/29_09_newdata8_rgbifndvi_datatran_40_epoch


In [ ]:
#prediction
# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'Tanuja_HKH'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

BANDS = opticalBands 

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 128
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

128

In [ ]:
# Use Sentinnel2 surface reflectance data for predictors.
l8sr = ee.ImageCollection('COPERNICUS/S2_SR')
BANDS = ['B2','B3','B4','B8','B11']

#cloud masking function of sentinnel data
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)
  mask = (qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000)

image = l8sr.filterDate('2019-01-01', '2019-1-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds).median()
image = image.unmask(0)

nir = image.select('B8');
red = image.select('B4');
        
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI');
image = image.addBands(ndvi);

image = image.select(['B2','B3','B4','B8','B11','NDVI'])
image.bandNames().getInfo()

['B2', 'B3', 'B4', 'B8', 'B11', 'NDVI']

In [ ]:
#image export function
def doExport(out_image_base, kernel_buffer, region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    #image = image.select(BANDS),
    image = image,
    #image = image,
    description = out_image_base,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 10,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
BANDS = ['B2','B3','B4','B8','B11','NDVI']

In [ ]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/tanujashrestha' # INSERT YOUR FOLDER HERE.
FOLDER = 'HKH/Tanuja'
# Base file name to use for TFRecord files and assets.

bj_image_base = 'Kathmandu'

#Note the above file is inside the 'urban256v3' folder

# Half this will extend on the sides of each patch.
bj_kernel_buffer = [128, 128]


bj_region = ee.Geometry.Polygon( #for Kathmandu
        [[[85.19264036381233,27.585543326088402],
          [85.49476438724983,27.585543326088402],
          [85.19264036381233,27.806237838670395],
          [85.19264036381233,27.585543326088402]]], None, False)


In [ ]:
# Run the export.
doExport(bj_image_base, bj_kernel_buffer, bj_region)

Running image export to Cloud Storage...
Image export completed.


In [ ]:
# Run the prediction.
doPrediction(bj_image_base, user_folder, bj_kernel_buffer,bj_region)